# Setup

## Install Dependencies

In [1]:
!pip3 install ./pal

Processing ./pal
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pal: filename=pal-0.1-py3-none-any.whl size=19902 sha256=b57bb049f80f05da968f82596ca6c7c19dbc4b7e520ee776e18a8e928899ccf7
  Stored in directory: /private/var/folders/8d/b6tnc2p90l548qg78nc_k0_m0000gn/T/pip-ephem-wheel-cache-8m6e9v6m/wheels/fc/96/b9/2d39b0564063a661921e159ff6fdb50d8eb7e8bdd95704e9bf
Successfully built pal
  Attempting uninstall: pal
    Found existing installation: pal 0.1
    Uninstalling pal-0.1:
      Successfully uninstalled pal-0.1


## Configure Openai API

In [2]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
# @param {type:"string"}
openai.api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4.1-nano"  # @param {type:"string"}

# Tracking of Objects

In [3]:
import copy
import json
import argparse
import tqdm
import re
import openai
import os
from dotenv import load_dotenv
from pal import interface
from pal.prompt import tracking_objects_prompt

In [4]:
DATA_PATH = "datasets/tracking_shuffled_objects_three_objects.json"
OUTPUT_PATH = "eval_results/tracking_objects_PAL_gpt-4.1-nano.jsonl"

In [5]:
examples = json.load(open(DATA_PATH))["examples"]
print(examples)

[{'input': 'Alice, Bob, and Claire are friends and avid readers who occasionally trade books. At the start of the semester, they each buy one new book: Alice gets Ulysses, Bob gets Frankenstein, and Claire gets Lolita.\nAs the semester proceeds, they start trading around the new books. First, Claire and Bob swap books. Then, Bob and Alice swap books. Finally, Claire and Bob swap books. At the end of the semester, Bob has\nOptions:\n(A) Ulysses\n(B) Frankenstein\n(C) Lolita', 'target': '(B)'}, {'input': 'Alice, Bob, and Claire are friends and avid readers who occasionally trade books. At the start of the semester, they each buy one new book: Alice gets Hound of the Baskervilles, Bob gets The Odyssey, and Claire gets Catch-22.\nAs the semester proceeds, they start trading around the new books. First, Claire and Alice swap books. Then, Bob and Claire swap books. Finally, Alice and Bob swap books. At the end of the semester, Alice has\nOptions:\n(A) Hound of the Baskervilles\n(B) The Odyss

In [6]:
itf = interface.ProgramInterface(
    stop="\n\n\n", get_answer_symbol="answer", verbose=True
)

In [7]:
append = False

In [8]:
if append:
    lines = open(OUTPUT_PATH).readlines()
    num_skip_exps = len(lines)
    scores = [x["score"] for x in map(json.loads, lines)]
else:
    num_skip_exps = 0
    scores = []

with open(OUTPUT_PATH, "a" if append else "w") as f:
    pbar = tqdm.tqdm(
        examples[num_skip_exps:], initial=num_skip_exps, total=len(examples)
    )
    for x in pbar:
        question = x["input"]
        result = copy.copy(x)

        try:
            ans = itf.run(
                tracking_objects_prompt.TRACKING_OBJECTS_PROMPT.format(question=question)
            )
            # ans = "Hound of the Baskervilles"
            options = re.findall(r"\(([A-C])\)\s([^\n]+)", x["input"])
            ans_str = ""

            for option, text in options:
                if text.strip() == ans.strip():
                    ans_str = f"({option})"
                    break
            result["answer_str"] = ans_str
        except:
            ans_str = ""

        if ans_str.strip() == x["target"].strip():
            score = 1
        else:
            score = 0

        scores.append(score)

        result["score"] = score
        # result['generation'] = itf.history
        f.write(json.dumps(result) + "\n")

        itf.clear_history()
        f.flush()
print(f"Accuracy - {sum(scores) / len(scores)}")

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 1/250 [00:03<14:53,  3.59s/it]

["# Initial possessions in a dictionary\nbooks = dict()\nbooks['Alice'] = 'Ulysses'\nbooks['Bob'] = 'Frankenstein'\nbooks['Claire'] = 'Lolita'\n\n# Claire and Bob swap books\nbooks['Claire'], books['Bob'] = books['Bob'], books['Claire']\n# Bob and Alice swap books\nbooks['Bob'], books['Alice'] = books['Alice'], books['Bob']\n# Claire and Bob swap books\nbooks['Claire'], books['Bob'] = books['Bob'], books['Claire']\nbob_book = books['Bob']\nanswer = bob_book"]


  1%|          | 2/250 [00:08<16:51,  4.08s/it]

["# Initial possessions in a dictionary\nbooks = dict()\nbooks['Alice'] = 'Hound of the Baskervilles'\nbooks['Bob'] = 'The Odyssey'\nbooks['Claire'] = 'Catch-22'\n\n# Claire and Alice swap books\nbooks['Alice'], books['Claire'] = books['Claire'], books['Alice']\n# Bob and Claire swap books\nbooks['Bob'], books['Claire'] = books['Claire'], books['Bob']\n# Alice and Bob swap books\nbooks['Alice'], books['Bob'] = books['Bob'], books['Alice']\nalice_book = books['Alice']\nanswer = alice_book"]


...

["# Initial partners in a dictionary\npartners = dict()\npartners['Alice'] = 'Rodrigo'\npartners['Bob'] = 'Jamie'\npartners['Claire'] = 'Lola'\n\n# Claire and Alice switch partners\npartners['Alice'], partners['Claire'] = partners['Claire'], partners['Alice']\n# Bob and Claire switch partners\npartners['Bob'], partners['Claire'] = partners['Claire'], partners['Bob']\n# Claire and Alice switch partners\npartners['Claire'], partners['Alice'] = partners['Alice'], partners['Claire']\nalice_partner = partners['Alice']\nanswer = alice_partner"]


100%|█████████▉| 249/250 [43:01<00:05,  5.80s/it]

["# Initial positions in a dictionary\npositions = dict()\npositions['Alice'] = 'right winger'\npositions['Bob'] = 'cheerleader'\npositions['Claire'] = 'striker'\n\n# Alice and Claire trade positions\npositions['Alice'], positions['Claire'] = positions['Claire'], positions['Alice']\n# Claire and Bob trade positions\npositions['Claire'], positions['Bob'] = positions['Bob'], positions['Claire']\n# Bob and Alice trade positions\npositions['Bob'], positions['Alice'] = positions['Alice'], positions['Bob']\nalice_position = positions['Alice']\nanswer = alice_position"]


100%|██████████| 250/250 [43:07<00:00, 10.35s/it]

["# Initial positions in a dictionary\npositions = dict()\npositions['Alice'] = 'right winger'\npositions['Bob'] = 'cheerleader'\npositions['Claire'] = 'striker'\n\n# Alice and Claire trade positions\npositions['Alice'], positions['Claire'] = positions['Claire'], positions['Alice']\n# Claire and Bob trade positions\npositions['Claire'], positions['Bob'] = positions['Bob'], positions['Claire']\n# Bob and Alice trade positions\npositions['Bob'], positions['Alice'] = positions['Alice'], positions['Bob']\nclaires_position = positions['Claire']\nanswer = claires_position"]
Accuracy - 0.612


In [9]:
print(f"Accuracy - {sum(scores) / len(scores)}")

Accuracy - 0.612
